In [6]:
!pip3 install tabula-py &> /dev/null
!pip3 install tabulate &> /dev/null
!pip3 install pdfplumber &> /dev/null
!pip3 install camelot-py &> /dev/null
!pip3 install yahoo_fin

  Using cached yahoo_fin-0.8.9.1-py3-none-any.whl (10 kB)
  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached feedparser-6.0.8-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Using cached pyquery-1.4.3-py3-none-any.whl (22 kB)
  Using cached pyppeteer-1.0.2-py3-none-any.whl (83 kB)
  Using cached parse-1.19.0.tar.gz (30 kB)
  Using cached fake-useragent-0.1.11.tar.gz (13 kB)
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
     |████████████████████████████████| 96 kB 1.0 MB/s eta 0:00:01
  Using cached pyee-8.2.2-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 128 kB 291 kB/s eta 0:00:01
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
Using legacy 'setup.py install' for fake-useragent, since package 'wheel' is not installed.
Using legacy 'setup.py install' for parse, since package 'wheel' is not installed.
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed

In [7]:

import requests
import numpy as np
import pandas as pd
import pdfplumber
from io import BytesIO
import re
from IPython.display import display, HTML
import yahoo_fin.stock_info as si

In [8]:
"""
Read SMIC quarterly management reports and 
extract revenue by technology, revenue by platform,
revenue by geography, and capex.
"""
def parse_smic(url):
    rq = requests.get(url)
    pdf = pdfplumber.open(BytesIO(rq.content))
    smic_text = ""
    for i in range(10):
        #try-catch here
        smic_text += pdf.pages[i].extract_text()
    tech_df = clean_smic_tech_robust(umc_text)
    segment_df = clean_smic_seg(umc_text)
    geo_df = clean_umc_geo(umc_text)
    capex_df = "INSERT CAPEX INFO HERE"
    return {'tech': tech_df, 'segment': segment_df, 'geo': geo_df, 'capex' : capex_df}

In [9]:
url = "https://smic.shwebspace.com/uploads/qe/201101271723318_en.pdf"

In [10]:
rq = requests.get(url)
pdf = pdfplumber.open(BytesIO(rq.content))
smic_text = ""
for i in range(10):
    #try-catch here
    smic_text += pdf.pages[i].extract_text()

In [24]:
def clean_smic_tech_robust(smic_text):
    digits = "\s+(\dQ\d+)\s+(\dQ\d+)\s+(\dQ\d+)"
    oldHeader = "Total\s+(?:\\n)?Wafer\s+(?:\\n)?Revenues"
    try:
        techCols = np.asarray(re.findall(f"(By Technology|{oldHeader}).*?{digits}", smic_text)[0])
    except: 
        techCols = np.array("Tech")
        techCols = np.append(techCols, np.asarray(re.findall(f"{digits}", smic_text)[0]))
    techArray = []
    digit_re = "\d+(?:\.\d+)(?:\s+)?(?:n|µ|m)m"
    slash = "\d+(?:\.\d+)?\/\d+(?:\.\d+)?\s+(?:n|µ|m)m"
    above = "\d+(?:\.\d+)?(?:u|n)m\s+and\s+above"
    numbers = "\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)"
    techRegex = f"({digit_re}|{slash}|{above}){numbers}"
    for i in re.finditer(techRegex, smic_text):
        techArray.append(list(i.groups()))
    techDF = pd.DataFrame(techArray, columns=techCols)
    firstColName = techDF.columns[0]
    print("firstColName")
    techDF = techDF.drop_duplicates(subset=firstColName, keep="first")
    return techDF
    

In [25]:
clean_smic_tech_robust(smic_text)

firstColName


,By Technology,2Q08,1Q08,2Q07
0,0.09mm,18.4%,19.8%,22.0%
1,0.15mm,2.1%,4.2%,1.2%
2,0.18mm,37.7%,32.1%,30.8%
3,0.25mm,0.6%,0.5%,0.7%
4,0.35mm,18.3%,18.4%,12.3%


In [ ]:
def clean_smic_geo(smic_text):
    umcRegions = []
    digits = "\s+(\d+%)\s+(\d+%)\s+(\d+%)"
    regionCols = np.asarray(re.findall(r"(Net Revenue by Geography|By Geography)\s+(\d+Q\d+)\s+(\d+Q\d+)\s+(\d+Q\d+)", tsmc_text)[0])
    NorthAm = np.asarray(re.findall(f"(North America){digits}", smic_text)[0])
    umcRegions.append(NorthAm)
    China = np.asarray(re.findall(f"(China){digits}", smic_text)[0])
    umcRegions.append(China)
    try:
        EMEA = np.asarray(re.findall(f"(EMEA){digits}", smic_text)[0])
        umcRegions.append(EMEA)
    except IndexError:
        pass
    try:
        europe = np.asarray(re.findall(f"(Europe){digits}", tsmc_text)[0])
        umcRegions.append(europe)
    except IndexError:
        pass
    try: 
        APAC = np.asarray(re.findall(f"(Asia Pacific){digits}", tsmc_text)[0])
        umcRegions.append(APAC)
    except IndexError:
        pass
    Japan = np.asarray(re.findall(f"(Japan){digits}", tsmc_text)[0])
    umcRegions.append(Japan)
    geoDF = pd.DataFrame(umcRegions, columns=regionCols)
    return geoDF